# Face Verification using CNN





In [ ]:
#Used libraries
import os
import numpy as np
from PIL import Image
import torchvision.transforms as transforms
import torch
import torchvision   
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import models
import time

### Data

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
#Copy data file to colab and extract it
!cp "/gdrive/My Drive/HW4_Data.rar" "HW4_Data.rar"
!unrar x "HW4_Data.rar"

In [ ]:
num_workers = 8
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_dataset = torchvision.datasets.ImageFolder(root='HW4_Data/classification_data/train_data/', 
                                                 transform=transform)
train_dataloader = DataLoader(train_dataset, batch_size=32, 
                                               shuffle=True, num_workers=num_workers)

val_dataset = torchvision.datasets.ImageFolder(root='HW4_Data/classification_data/val_data/', 
                                                 transform=transform)
val_dataloader = DataLoader(val_dataset, batch_size=32, 
                                               shuffle=True, num_workers=num_workers)

test_dataset = torchvision.datasets.ImageFolder(root='HW4_Data/classification_data/test_data/', 
                                                 transform=transform)
test_dataloader = DataLoader(test_dataset, batch_size=32, 
                                               shuffle=False, num_workers=num_workers)
print(len(train_dataset))

380638


### Model

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = torch.hub.load('pytorch/vision:v0.6.0', 'resnet152', pretrained=True)

Downloading: "https://github.com/pytorch/vision/archive/v0.6.0.zip" to /root/.cache/torch/hub/v0.6.0.zip
Downloading: "https://download.pytorch.org/models/resnet152-b121ed2d.pth" to /root/.cache/torch/hub/checkpoints/resnet152-b121ed2d.pth


In [ ]:
model.fc = nn.Linear(in_features=2048, out_features=4000, bias=True)
print(model)
model.to(device)
# optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.005, momentum=0.9)
# loss function
criterion = nn.CrossEntropyLoss()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

### Training Model

In [ ]:
def train(model, data_loader, test_loader, nextEpoch = 0, task='Classification'):
    model.train()
    for epoch in range(numEpochs):
        start_time = time.time()
        avg_loss = 0.0
        for batch_num, (feats, labels) in enumerate(data_loader):
            feats, labels = feats.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(feats)
            loss = criterion(outputs, labels.long())
            loss.backward()
            optimizer.step()
            
            avg_loss += loss.item()
            everyB = 200
            if batch_num % everyB == (everyB-1):
                print('Epoch: {}\tBatch: {}\tAvg-Loss: {:.4f}'.format(epoch+1+nextEpoch, batch_num+1, avg_loss/everyB))
                avg_loss = 0.0    
            
            torch.cuda.empty_cache()
            del feats
            del labels
            del loss
        
        if task == 'Classification':
            val_loss, val_acc = test_classify(model, test_loader)
            train_loss, train_acc = test_classify(model, data_loader)
            print('Train Loss: {:.4f}\tTrain Accuracy: {:.4f}\tVal Loss: {:.4f}\tVal Accuracy: {:.4f}'.
                  format(train_loss, train_acc, val_loss, val_acc))
        else:
            test_verify(model, test_loader)
        
        end_time = time.time()
        print('Time: ',end_time - start_time, 's')

        # torch.save({
        #     'epoch': (epoch+nextEpoch),
        #     'model_state_dict': model.state_dict(),
        #     'optimizer_state_dict': optimizer.state_dict()}, '/gdrive/My Drive/modelRe'+(str)(epoch+nextEpoch)+'.pt')
        torch.save({
            'epoch': (epoch+nextEpoch),
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict()}, '/gdrive/My Drive/modelRe'+(str)(epoch+nextEpoch)+'.pt')

def test_classify(model, test_loader):
    model.eval()
    test_loss = []
    accuracy = 0
    total = 0

    for batch_num, (feats, labels) in enumerate(test_loader):
        feats, labels = feats.to(device), labels.to(device)
        outputs = model(feats)
              
        _, pred_labels = torch.max(F.softmax(outputs, dim=1), 1)
        pred_labels = pred_labels.view(-1)
        
        everyB = 400
        # if batch_num % everyB == (everyB-1):
        #     print('Test Batch: {}'.format(batch_num+1))
        
        loss = criterion(outputs, labels.long())
        accuracy += torch.sum(torch.eq(pred_labels, labels)).item()
        total += len(labels)
        test_loss.extend([loss.item()]*feats.size()[0])
        del feats
        del labels
    
    model.train()
    return np.mean(test_loss), accuracy/total


def test_verify(model, test_loader):
    raise NotImplementedError

In [ ]:
#Load previous training status
checkpoint = torch.load('/gdrive/My Drive/modelRes.pt')
#checkpoint = torch.load('/gdrive/My Drive/modelRe14.pt')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
e = checkpoint['epoch']

In [ ]:
numEpochs = 5
print(len(train_dataset))
train(model, train_dataloader, val_dataloader, e)

380638
Epoch: 12	Batch: 200	Avg-Loss: 0.0046
Epoch: 12	Batch: 400	Avg-Loss: 0.0042
Epoch: 12	Batch: 600	Avg-Loss: 0.0035
Epoch: 12	Batch: 800	Avg-Loss: 0.0042
Epoch: 12	Batch: 1000	Avg-Loss: 0.0043
Epoch: 12	Batch: 1200	Avg-Loss: 0.0040
Epoch: 12	Batch: 1400	Avg-Loss: 0.0042
Epoch: 12	Batch: 1600	Avg-Loss: 0.0040
Epoch: 12	Batch: 1800	Avg-Loss: 0.0038
Epoch: 12	Batch: 2000	Avg-Loss: 0.0044
Epoch: 12	Batch: 2200	Avg-Loss: 0.0038
Epoch: 12	Batch: 2400	Avg-Loss: 0.0043
Epoch: 12	Batch: 2600	Avg-Loss: 0.0040
Epoch: 12	Batch: 2800	Avg-Loss: 0.0035
Epoch: 12	Batch: 3000	Avg-Loss: 0.0039
Epoch: 12	Batch: 3200	Avg-Loss: 0.0038
Epoch: 12	Batch: 3400	Avg-Loss: 0.0032
Epoch: 12	Batch: 3600	Avg-Loss: 0.0038
Epoch: 12	Batch: 3800	Avg-Loss: 0.0037
Epoch: 12	Batch: 4000	Avg-Loss: 0.0035
Epoch: 12	Batch: 4200	Avg-Loss: 0.0035
Epoch: 12	Batch: 4400	Avg-Loss: 0.0035
Epoch: 12	Batch: 4600	Avg-Loss: 0.0033
Epoch: 12	Batch: 4800	Avg-Loss: 0.0042
Epoch: 12	Batch: 5000	Avg-Loss: 0.0036
Epoch: 12	Batch: 5200	

### Results after 15 epochs of training

In [ ]:
train_loss, train_acc = test_classify(model, train_dataloader)
val_loss, val_acc = test_classify(model, val_dataloader)
test_loss, test_acc = test_classify(model, test_dataloader)
print('Train Loss: {:.4f}\tTrain Accuracy: {:.4f}\tVal Loss: {:.4f}\tVal Accuracy: {:.4f}'.
                  format(train_loss, train_acc, val_loss, val_acc))
print('Test Loss: {:.4f}\tTest Accuracy: {:.4f}'.
                  format(test_loss, test_acc))

Train Loss: 0.0002	Train Accuracy: 1.0000	Val Loss: 0.6717	Val Accuracy: 0.8738
Test Loss: 0.6634	Test Accuracy: 0.8776


### Face Verification

In [ ]:
#Read text file
def readFile():
  f = open('HW4_Data/verification_pairs_val.txt','r')
  pairs = []
  values = []
  for line in f:
    a = line.strip().split(' ')
    pairs += [[a[0],a[1]]]
    values += a[2]

  f.close()
  return pairs, values

pairs, values =readFile()

In [ ]:
trainedModel = torch.load('/gdrive/My Drive/modelRes.pt')
#trainedModel = torch.load('/gdrive/My Drive/modelRe14.pt')
model.load_state_dict(trainedModel['model_state_dict'])

<All keys matched successfully>

In [ ]:
#Remove last layer
model.fc = nn.Sequential()
model.eval()
model.to(device)
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
def verification(model, pairs, values):
  transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
  ])

  scores = []
  for i in range(len(pairs)):
    if i%600 ==0:
      print(i)
    img1 = Image.open('HW4_Data/'+pairs[i][0])
    img1 = transform(img1)
    img1 = img1.unsqueeze(0)
    img1 = img1.to(device)

    img2 = Image.open('HW4_Data/'+pairs[i][1])
    img2 = transform(img2)
    img2 = img2.unsqueeze(0)
    img2 = img2.to(device)

    out1 = model(img1)
    out2 = model(img2)
    y = torch.nn.functional.cosine_similarity(out1[0], out2[0], dim=-1, eps=1e-8)
    scores += [y.item()]

  return scores

scores = verification(model, pairs, values)

0
600
1200
1800
2400
3000
3600
4200
4800
5400
6000
6600
7200
7800
8400


In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
auc = roc_auc_score(values, scores)
print("AUC score:", auc)

AUC score: 0.9608603385403897


In [ ]:
results = []
for i in range(len(scores)):
  if scores[i] > 0.6:
    results += [1]
  else:
    results += [0]

In [ ]:
for i in range(len(values)):
  values[i] = (int)(values[i])

acc = accuracy_score(values, results)
print("Verification Accuracy (with choosing threshold = 0.6):", acc)

Verification Accuracy (with choosing threshold = 0.6): 0.8999432140829075
